In [2]:
import pandas as pd
import numpy as np
import random
from pyvi import ViTokenizer, ViPosTagger
from pyvi import ViUtils
np.random.seed(1)
random.seed(1)

TRAININFO = "/media/DATA/zalo-hit-song-prediction/csv/train_info.tsv"
TRAINRANK =  "/media/DATA/zalo-hit-song-prediction/csv/train_rank.csv"
TESTINFO = "/media/DATA/zalo-hit-song-prediction/csv/test_info.tsv"
Track_info = "/media/DATA/zalo-hit-song-prediction/csv/all_track_info.csv"
Audio_info = "/media/DATA/zalo-hit-song-prediction/csv/all_track_audio_features.csv"
df_i = pd.read_csv(TRAININFO, delimiter='\t',encoding='utf-8')
df_r = pd.read_csv(TRAINRANK)
df_i_train = df_i.merge(df_r, left_on='ID', right_on='ID')
df_i_train["dataset"] = "train"

df_i_test = pd.read_csv(TESTINFO, delimiter='\t',encoding='utf-8')
df_i_test["label"] = np.nan
df_i_test["dataset"] = "test"

df = pd.concat([df_i_train, df_i_test])
df_track_info = pd.read_csv(Track_info)
df = df.merge(df_track_info, left_on='ID', right_on='ID')
df_audio_features = pd.read_csv(Audio_info)
df =df.merge(df_audio_features,left_on="ID",right_on="ID", how="left")
df = df[['ID','title','artist_name','lyric','label','dataset']]
df.head()

with open("/media/DATA/zalo-hit-song-prediction/csv/stopwords/vietnamese-stopwords.txt", 'r') as f:
    filecontent=f.readlines()
stopwords = list(set([f.strip() for f in filecontent]))


stopwords_tokenized = list(set([ViTokenizer.tokenize(stopword.lower()) for stopword in stopwords]))
set([ViTokenizer.tokenize(word) for word in stopwords_tokenized]) - set(stopwords_tokenized)
stopwords_tokenized[0]

In [3]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
def remove_stopwords(string):
    new_string = [word for word in string if word not in stopwords]
    return new_string
df.dropna(subset=['lyric'], inplace=True) # drop songs with no lyrics
df_train = df[df.dataset=='train']
df_test = df[df.dataset=='test']
df_train = df_train.drop_duplicates(subset='lyric', keep='first')

[nltk_data] Downloading package punkt to /home/ben/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [72]:
from sklearn.pipeline import Pipeline, FeatureUnion, make_union
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import HashingVectorizer

'''
def encode_column(df, column_name, to_n_columns=12):
    
    hasher=category_encoders.hashing.HashingEncoder()
    #encoded=hasher.fit_transform(np.expand_dims(df["university_name"].values, axis=-1),verbose=True, cols=None, drop_invariant=True, return_df=True, hash_method='sha1')
    tmp=pd.DataFrame(np.expand_dims(df["university_name"].values, axis=-1))
    encoded_df=hasher.hashing_trick(tmp, hashing_method='md5', N=to_n_columns, make_copy=False)
    
    encoded_col_names=list(encoded_df.columns.values)
    tmp=df.copy()
    tmp["index"]=tmp.index
    tmp=tmp.reset_index()
    df=pd.merge(tmp, encoded_df, how='left', left_index=True, right_index=True, sort=False,
             suffixes=('_left', '_right'),validate="one_to_one")
    
    print(type(encoded_col_names))
    df.set_index("index",inplace=True)
    return df, encoded_col_names
'''
class TextColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


# generate features - in this case just one.
feature = Pipeline([
                ('selector_1', TextColumnSelector(key='lyric')),
                ('vect', HashingVectorizer(n_features = 2**4, 
                                           tokenizer=ViTokenizer.tokenize, 
                                           lowercase=True, 
                                           stop_words=stopwords_tokenized,
                                           norm='l2')),
                ]
                #('vect', CountVectorizer(analyzer='word', tokenizer=str.split, stop_words=stopwords)),
                #('tfidf', TfidfTransformer(use_idf=True, sublinear_tf=True))
                )
# Another option:
##hasher=category_encoders.hashing.HashingEncoder()
##encoded_df=hasher.hashing_trick(tmp, hashing_method='md5', N=to_n_columns, make_copy=False)
#vectorizer = HashingVectorizer(n_features = 2**4, tokenizer=word_tokenize, lowercase=True, stop_words=stopwords,  norm='l2')
#vectors = vectorizer.fit_transform(df_train.lyric)

feats_list=[('text', feature)]
feats = FeatureUnion(feats_list)


'từ giờ'

In [85]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline, FeatureUnion, make_union
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
def rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions))
rmse_scorer = make_scorer(rmse, greater_is_better=False)


#model=LogisticRegression(class_weight="balanced",penalty='l1', random_state=1)
#model_lc="classifier__C"
#values=[719.686, ] # np.logspace(-4, 4, 8) # 0.001 to 1e4
#http://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html 
model=SVR(kernel="rbf", C=np.logspace(-4, 4, 8))
model_lc="classifier__gamma"
values = np.logspace(-4, 4, 4)
model_lc_2 = "classifier__C"
values2 =  np.logspace(-4, 4, 8)[0:4]


model_pipeline = Pipeline([
    ('features',feats),
    ('classifier', model),
])

#param_range = np.logspace(1, 10, 5)

parameters = {'features__text__vect__n_features': np.array([1e3,1e4,1e5]).astype('int64'),
              'features__text__vect__ngram_range': [(1,1)],# [(1,1), (1, 2),], # try bigrams or unigrams
              model_lc: values,
              model_lc_2:values2,
             }

skf = StratifiedKFold(n_splits=10, random_state=99999)
gs_clf = RandomizedSearchCV(model_pipeline, 
                      parameters, 
                      scoring=rmse_scorer, 
                      cv=skf,
                      n_jobs=-1,
                      return_train_score=False,
                      error_score='raise',
                      n_iter=10,
                      verbose=10
                    )

#gs_clf.get_params().keys()
#[i for i in gs_clf.get_params().keys()]
gs_clf.fit(df_train, df_train.label)

print(gs_clf.cv_results_['mean_test_score'][gs_clf.best_index_], "+/-" ,gs_clf.cv_results_['std_test_score'][gs_clf.best_index_])
print(gs_clf.best_score_, gs_clf.best_params_)
pd.DataFrame(gs_clf.cv_results_).to_csv('svr_lyric_unigram2.csv')

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   22.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  96 out of 100 | elapsed:  4.2min remaining:   10.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.4min finished
/media/DATA/anaconda3/envs/thriai/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated to

-2.920596703520487 +/- 0.0930542621402546
-2.920596703520487 {'features__text__vect__ngram_range': (1, 1), 'features__text__vect__n_features': 100000, 'classifier__gamma': 21.54434690031882, 'classifier__C': 0.2682695795279725}


array([1.00000000e-04, 1.38949549e-03, 1.93069773e-02, 2.68269580e-01])

In [78]:
pd.DataFrame(gs_clf.cv_results_).to_csv('svr_lyric_unigram.csv')

In [80]:
pd.DataFrame(gs_clf.cv_results_)[['param_classifier__C','param_classifier__gamma', 'param_features__text__vect__n_features','mean_test_score', 'std_test_score']].sort_values('mean_test_score')

,param_classifier__C,param_classifier__gamma,param_features__text__vect__n_features,mean_test_score,std_test_score
1,0.26827,0.26827,10000,3.011608,0.008735
2,0.26827,10000,10000,3.025377,0.004742
7,0.0001,51.7947,10000,3.027342,0.001695
3,0.26827,0.0013895,10000,3.027458,0.001704
8,3.72759,10000,10000,3.056874,0.037078
9,10000,10000,10000,3.062024,0.040747
5,51.7947,10000,10000,3.062024,0.040749
4,51.7947,719.686,10000,3.085559,0.046583
0,10000,719.686,10000,3.090328,0.047625
6,719.686,51.7947,10000,3.546580,0.146699


In [52]:
import sklearn
def rmse(targets, predictions):
    return np.sqrt(mean_squared_error(targets, predictions))
rmse_scorer = make_scorer(rmse, greater_is_better=False)

dummy_clf = sklearn.dummy.DummyClassifier(strategy='stratified', random_state=99999)
skf = StratifiedKFold(n_splits=10, random_state=99999)
from sklearn.model_selection import cross_validate
cv_pred = cross_validate(dummy_clf, df_train['lyric'], df_train.label, scoring=rmse_scorer, cv=skf)
cv_pred['test_score'].mean(),  '+/-', cv_pred['test_score'].std()

(4.277847802266441, '+/-', 0.08664532999630303)

In [32]:
trainDF['char_count'] = trainDF['text'].apply(len)
trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)

array([1.e+09, 1.e+10])